In [ ]:
import datetime
from functools import partial
import math
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
import time

import cv2
import tensorflow as tf
from tensorflow.keras import backend as K


In [ ]:
#jupyter 中开启该选项，否则不执行
%matplotlib inline

## 配置超参数

In [ ]:
INPUT_PATCH_SHAPE = (16, 16, 3)             # shape of input image patch: (W, H, C)  
INPUT_PATCH_SIZE = 16                       # equal to W or H of INPUT_PATCH_SHAPE
BATCH_SIZE = 256                            # 
BUFFER_SIZE = 60000                         # related to shuffle
EPOCHS = 30                                 # try different EPOCHS and supervise in TensorBoard
VALIDITION_SPLIT = 0.15                     # float between 0 and 1 : ratio of validation set to data set
WEIGHT_DECAY = 1e-4                         # 

OUTPUT_FOLDER = '.\\model_output'           # the path saved ours models and logs
IMG_DIR = '.\\trains'                       # training datasets path

if not os.path.exists(OUTPUT_FOLDER):
    os.mkdir(OUTPUT_FOLDER)
    
LOG_DIR = os.path.join(OUTPUT_FOLDER, 'logs_{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)


In [ ]:
# INFO == 0：    | output : INFO + WARNING + ERROR + FATAL | ignore : nothing  
# WARNING == 1:  | output : WARNING + ERROR + FATAL        | ignore : INFO
# ERROR == 2:    | output : ERROR + FATAL                  | ignore : INFO + WARNING
# FATAL == 3:    | output : FATAL                          | ignore : INFO + WARNING + ERROR
# 不同的值设置的是基础的log信息（base_loging），运行时会输出base等级及之上的信息。
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## 选择指定显卡及自动调用显存

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')# 列出所有可见显卡
print("All the available GPUs:\n", physical_devices)
if physical_devices:
    gpu = physical_devices[0] # 显示第一块显卡
    tf.config.experimental.set_memory_growth(gpu, True) # 根据需要自动增长显存
    tf.config.experimental.set_visible_devices(gpu, 'GPU') # 只选择第一块

## 数据准备

In [ ]:
help(max)

In [ ]:
def create_dataset(img_dir, num_t, patch_size = INPUT_PATCH_SIZE, validation_split= VALIDITION_SPLIT):
    """
    input:
    img_dir: dir of haze-free images
    num_t: a haze_free image patch generate num_t hazy image patches
    patch_size: side length of image patch
    validation_split: ratio of validation set to data set
    
    output:
    """
     
   
    img_path = os.listdir(img_dir)
    
    random.shuffle(img_path)
    
    x_train = []
    
    y_train = []
    

    for image_name in img_path:
        
        fullname = os.path.join(img_dir, image_name)
        
        img = cv2.imread(fullname)
        
        w,h,_ = img.shape
        
        num_w = int(w / patch_size)
        
        num_h = int(h / patch_size)
        
        for i in range(num_w):
            
            for j in range(num_h):
                
                free_patch = img[0+i*patch_size:patch_size+i*patch_size, 0+j*patch_size:patch_size+j*patch_size, :]
                
                for k in range(num_t): 
                   
                    t = random.random()                   
                    
                    hazy_patch = t * free_patch / np.float(255)  + (1 - t)         # Note that the image pixel value has been normalized here
                    
                    x_train.append(hazy_patch)
                    
                    y_train.append(t)
                    

    x_train = np.array(x_train)
    
    y_train = np.array(y_train)

    # to make the dimension of y_train equal with x_train's
    y_train = np.expand_dims(y_train,axis=-1)
    
    y_train = np.expand_dims(y_train,axis=-1)
    
    y_train = np.expand_dims(y_train,axis=-1)
    

    total_num = x_train.shape[0]
    
    train_num = int(total_num*(1-validation_split))
    
    x_t = x_train[:train_num]
    
    x_val = x_train[train_num:]
    
    y_t = y_train[:train_num]
    
    y_val = y_train[train_num:]
    
    print('The shape of x_train: ',x_t.shape)
    
    print('The shape of y_train: ',y_t.shape)
    
    print('The shape of x_validation: ',x_val.shape)
    
    print('The shape of y_validation: ',y_val.shape)
    
    return x_t, y_t, x_val, y_val

## 定义模型

In [ ]:
class MaxoutConv2D(tf.keras.layers.Layer):
    
    
    def __init__(self, **kwargs):
        
        super(MaxoutConv2D, self).__init__(**kwargs)
        
        
    def call(self, x):
        
        output = K.max(x, axis=-1, keepdims=True)
        
        return output
    
    
    def compute_output_shape(self, input_shape):
        
        input_height = input_shape[1]
        
        input_width = input_shape[2]
        
        output_height = input_height
        
        output_width = input_width
        
        return (input_shape[0], output_height, output_width, 1)
    
    
# metric r2
def r2(y_true, y_pred):
    
    return 1 - K.sum(K.square(y_pred - y_true))/K.sum(K.square(y_true - K.mean(y_true)))

In [ ]:
def dehaze_model(input_shape = INPUT_PATCH_SHAPE):
        
        
    initializer = tf.initializers.RandomNormal(stddev=0.001)           # you can try different initialization strategies
    
    kernel_regularizer=tf.keras.regularizers.l2(WEIGHT_DECAY)
    
    inputs = tf.keras.Input(shape=input_shape)
    
    # conv2d
    conv_11 = tf.keras.layers.Conv2D(filters=4, kernel_size=(1, 1), strides=(1, 1), padding='same',
                                  kernel_initializer=initializer)(inputs)
    
    conv_12 = tf.keras.layers.Conv2D(filters=4, kernel_size=(3, 3), strides=(1, 1), padding='same',
                                  kernel_initializer=initializer)(inputs)
    
    conv_13 = tf.keras.layers.Conv2D(filters=4, kernel_size=(5, 5), strides=(1, 1), padding='same',
                                  kernel_initializer=initializer)(inputs)
    
    conv_14 = tf.keras.layers.Conv2D(filters=4, kernel_size=(7, 7), strides=(1, 1), padding='same',
                                  kernel_initializer=initializer)(inputs)
    
    
    # Maxout layer output 16x16
    max_out_1 = MaxoutConv2D()(conv_11)

    max_out_2 = MaxoutConv2D()(conv_12)
    
    max_out_3 = MaxoutConv2D()(conv_13)
    
    max_out_4 = MaxoutConv2D()(conv_14)
    
    max_out = tf.keras.layers.Concatenate()([max_out_1, max_out_2, max_out_3, max_out_4])
    
    # Conv - BN - ReLU
    # conv2 input 16x16 output 14x14
    conv_2 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='valid',
                                  kernel_initializer=initializer)(max_out)
    
    conv_2 = tf.keras.layers.BatchNormalization()(conv_2)
    
    conv_2 = tf.keras.layers.ReLU()(conv_2)
    
    # Conv - BN - ReLU
    # conv3 input 14x14 output 12x12
    conv_3 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='valid',
                                  kernel_initializer=initializer)(conv_2)
    
    conv_3 = tf.keras.layers.BatchNormalization()(conv_3)
    
    conv_3 = tf.keras.layers.ReLU()(conv_3)
    
    # Conv - BN - ReLU
    # conv4 input 12x12 output 10x10
    conv_4 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='valid',
                                  kernel_initializer=initializer)(conv_3)
    
    conv_4 = tf.keras.layers.BatchNormalization()(conv_4)
    
    conv_4 = tf.keras.layers.ReLU()(conv_4)
    
    # 2x2 max pooling input 10x10 output 5x5 
    max_pool_1 = tf.keras.layers.MaxPool2D(pool_size=(2 ,2), strides=(2, 2))(conv_4)
    
    # conv5 input 5x5 output 1x1
    conv_5 = tf.keras.layers.Conv2D(filters=1, kernel_size=(5, 5), strides=(1, 1), padding='valid',
                                  kernel_initializer=initializer)(max_pool_1)
    
    outputs = tf.keras.layers.ReLU()(conv_5)
 
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model

## 模型训练及验证集loss和r2曲线可视化

In [ ]:
def plot_ls(history):
    plt.figure()
    N=np.arange(EPOCHS)
    plt.plot(N,history.history['loss'],label='train_loss')
    plt.scatter(N,history.history['loss'])
    plt.plot(N,history.history['val_loss'],label='val_loss')
    plt.scatter(N,history.history['val_loss'])
    plt.title('Training Loss on Our_dataset')
    plt.xlabel('Epoch #')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(LOG_DIR,'training_loss.png'))
    
def plot_r2(history):
    plt.figure()
    N=np.arange(EPOCHS)
    plt.plot(N,history.history['r2'],label='train_r2')
    plt.scatter(N,history.history['r2'])
    plt.plot(N,history.history['val_r2'],label='val_r2')
    plt.scatter(N,history.history['val_r2'])
    plt.title('Training r2 on Our_dataset')
    plt.xlabel('Epoch #')
    plt.ylabel('r2')
    plt.legend()
    plt.savefig(os.path.join(LOG_DIR,'training_r2.png'))

## 训练

In [ ]:
# rewrite
def train():
    
    K.clear_session()
       
    print("#" * 20)
    
    print("正在加载数据......")
    
    # preparing dataset
    x_train,y_train,x_val,y_val = create_dataset(img_dir=IMG_DIR, num_t=10)
    
    total_train_samples = x_train.shape[0]
    
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    
    val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    
    print("数据加载完毕。")
    
    print("#" * 20)
    
    # optimizer
    # use Adam with default parameters
    optimizer = tf.keras.optimizers.Adam()
    
    # load model
    model = dehaze_model()
    
    print("正在保存模型结构图......")
    
    tf.keras.utils.plot_model(model, to_file=os.path.join(LOG_DIR,'model.png'),
                             show_shapes=True, show_layer_names=True)
    
    print("模型结构图保存完毕。")
    
    print("#" * 20)
    
    # model compile
    model.compile(optimizer=optimizer,
                 loss='mse',
                 metrics=[r2])
    
    print("编译成功。")
    
    print("准备训练......")
    
    # Tensorboard 可视化训练过程
    # 训练过程或完成后，可以通过在命令行输入 tensorboard --logdir <TensorBoardLog的相对地址或绝对地址>
    # 注意查看打开的tensorboard的Data location一项地址是否指向上述地址。。。
    # 如果你发现Data location的地址不是你输入的地址，那通过给tensorboard --logdir <TensorBoardLog的相对地址或绝对地址>这一命令
    # 后加参数 --port:6005来解决（或其他没被占用的端口）
    
    tblog_dir = os.path.join(LOG_DIR, 'TensorBoardLog')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tblog_dir,
                                                         histogram_freq=1)
    
    # early stop strategy
    earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_r2', min_delta=0.0001,
                                                patience=2, verbose=True)
    
    history = model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=EPOCHS,
                        verbose = 1,        # only useful in interactive enviroment                        
                        callbacks=[tensorboard_callback, earlystop])
    
    # metrics are stateful. they accumulate values and return a cumulative
    # result when you call .result().
    # reset metrics before saving so that loaded model has same state,
    # since metric states are not preserved by Model.save_weights
    model.reset_metrics()
    
    model.save(os.path.join(LOG_DIR,'model.hdf5'))
    
    print("model.hdf5 saved")
    
    # save the image of loss and r2
    # this is just to let you have an intuitive feeling. go to tensorboard for specific training information
    plot_ls(history)
    
    plot_r2(history)
    
    print(history.history)

In [ ]:
if __name__ =="__main__":
    train()